<a href="https://colab.research.google.com/github/nibchee/data_science_machine_learning/blob/main/Fetaure_Selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [17]:
df=pd.read_csv('/content/WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [18]:
df.head(5)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [19]:
df['TotalCharges']=pd.to_numeric(df['TotalCharges'],errors='coerce')

In [20]:
df.drop(['customerID'],axis='columns',inplace=True)

In [21]:
#Dummy Encoding
df=pd.get_dummies(df,drop_first=True)

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 31 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   SeniorCitizen                          7043 non-null   int64  
 1   tenure                                 7043 non-null   int64  
 2   MonthlyCharges                         7043 non-null   float64
 3   TotalCharges                           7032 non-null   float64
 4   gender_Male                            7043 non-null   bool   
 5   Partner_Yes                            7043 non-null   bool   
 6   Dependents_Yes                         7043 non-null   bool   
 7   PhoneService_Yes                       7043 non-null   bool   
 8   MultipleLines_No phone service         7043 non-null   bool   
 9   MultipleLines_Yes                      7043 non-null   bool   
 10  InternetService_Fiber optic            7043 non-null   bool   
 11  Inte

In [24]:
#Considering 'Churn'-> y varible
#Other cols as x var
X=df.drop(['Churn_Yes'],axis=1)
y=df['Churn_Yes']

In [7]:
y

,Churn
0,No
1,No
2,Yes
3,No
4,Yes
...,...
7038,No
7039,No
7040,No
7041,Yes


In [44]:
#Split Data into training and test sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)

# New cell: Imputation and Scaling
from sklearn.preprocessing import StandardScaler

# Impute missing values in 'TotalCharges' of X_train and X_test
X_train['TotalCharges'] = X_train['TotalCharges'].fillna(X_train['TotalCharges'].mean())
X_test['TotalCharges'] = X_test['TotalCharges'].fillna(X_train['TotalCharges'].mean()) # Use X_train mean for test data

# Identify numerical columns for scaling
numerical_cols = ['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges']

# Initialize StandardScaler
scaler = StandardScaler()

# Create scaled copies of the dataframes
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

# Fit scaler only on the numerical columns of the training data
scaler.fit(X_train_scaled[numerical_cols])

# Transform both training and test data
X_train_scaled[numerical_cols] = scaler.transform(X_train_scaled[numerical_cols])
X_test_scaled[numerical_cols] = scaler.transform(X_test_scaled[numerical_cols])

In [39]:
#Apply recursive features eliminatoin(RFE) to select 5  best features
model=LogisticRegression(max_iter=1000) # Increased max_iter
rfe=RFE(model,n_features_to_select=5)

In [41]:
# The imputation for TotalCharges is now handled in the new scaling cell, so remove it from here

rfe=rfe.fit(X_train_scaled,y_train) # Use scaled data

NameError: name 'X_train_scaled' is not defined

In [42]:
#Get Selected Features
selected_features=X_train_scaled.columns[rfe.support_]
print(selected_features)

NameError: name 'X_train_scaled' is not defined

In [43]:
X_train_selected=X_train_scaled[selected_features]
X_test_selected=X_test_scaled[selected_features]

NameError: name 'X_train_scaled' is not defined

In [45]:
#Buld Model
model_orig=LogisticRegression(max_iter=1000)

# The imputation for TotalCharges in X_test is now handled in the new scaling cell, so remove it from here

# model_orig.fit(X_train_scaled,y_train) # Use scaled training data
# y_pred_orig=model_orig.predict(X_test_scaled) # Use scaled test data for prediction

In [ ]:
accuracy = accuracy_score(y_test, y_pred_orig)
print(f"Original Model Accuracy: {accuracy:.4f}")